In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import os

In [2]:
def load_images():
    path_train_b = "data/train/benign"
    path_train_m ="data/train/malignant"
    path_test_b = "data/test/benign"
    path_test_m = "data/test/malignant"
    paths = [path_train_b, path_train_m , path_test_b, path_test_m]
    
    x_train_b = []
    x_train_m = []
    x_test_b = []
    x_test_m = []
    x = [x_train_b, x_train_m, x_test_b, x_test_m]
    
    for i in range(len(paths)):
        for filename in os.listdir(paths[i]):
            img = cv2.imread(os.path.join(paths[i],filename))
            if img is not None:
                x[i].append(img)
    
    [x_train_b,x_train_m,x_test_b,x_test_m] = x
    return x_train_b,x_train_m,x_test_b,x_test_m
            


In [3]:
x_train_b,x_train_m,x_test_b,x_test_m = load_images()
x_train_b = np.array(x_train_b)
x_train_m = np.array(x_train_m)
x_test_b = np.array(x_test_b)
x_test_m = np.array(x_test_m)

x_train_b = x_train_b
x_train_m = x_train_m
x_test_b = x_test_b[0:50]
x_test_m = x_test_m[0:50]

In [4]:
#labels
y_train_b = np.zeros((x_train_b.shape[0],1), dtype = int)
y_train_m = np.ones((x_train_m.shape[0],1), dtype= int)
y_test_b = np.zeros((x_test_b.shape[0],1), dtype = int)
y_test_m = np.ones((x_test_m.shape[0],1), dtype = int)

#merging
x_train = np.concatenate((x_train_b,x_train_m), axis = 0)
y_train = np.concatenate((y_train_b,y_train_m), axis = 0)
x_test = np.concatenate((x_test_b,x_test_m), axis = 0)
y_test = np.concatenate((y_test_b,y_test_m), axis = 0)

X = np.concatenate((x_train,x_test), axis = 0)
Y = np.concatenate((y_train,y_test), axis = 0)


In [5]:


m_train = x_train.shape[0]
num_px = x_train.shape[1]
m_test = x_test.shape[0]

print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_x_orig shape: " + str(x_train.shape))
print ("train_y shape: " + str(y_train.shape))
print ("test_x_orig shape: " + str(x_test.shape))
print ("test_y shape: " + str(y_test.shape))

Number of training examples: 2637
Number of testing examples: 100
Each image is of size: (224, 224, 3)
train_x_orig shape: (2637, 224, 224, 3)
train_y shape: (2637, 1)
test_x_orig shape: (100, 224, 224, 3)
test_y shape: (100, 1)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, shuffle = True, random_state = 42)

#reshaping
X_train = X_train.reshape(X_train.shape[0], -1).T
X_test = X_test.reshape(X_test.shape[0], -1).T
Y_train = Y_train.T
Y_test = Y_test.T

#normalize
X_train = X_train/255
X_test = X_test/255

print ("train_x's shape: " + str(X_train.shape))
print ("test_x's shape: " + str(X_test.shape))
print( "train_y's shape" + str(Y_train.shape))
print ("test_y's shape" + str(Y_test.shape))

train_x's shape: (150528, 2189)
test_x's shape: (150528, 548)
train_y's shape(1, 2189)
test_y's shape(1, 548)


In [7]:
def initialize_parameters(n_x, n_h, n_y):
    
    np.random.seed(10)
    
    w1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    w2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))
   
    parameters = {"W1": w1,
                  "b1": b1,
                  "W2": w2,
                  "b2": b2}
    
    return parameters     



In [8]:
def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    
    return Z, cache

In [9]:
def sigmoid(z):
    
    a = 1/(1+np.exp(-z))
    cache = z
    
    return a, cache

In [10]:
def relu(z):
    
    a = np.maximum(0,z)
    
    assert(a.shape == z.shape)
    
    cache = z 
    return a, cache

In [11]:
def relu_backward(dA, cache):
   
    
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    return dZ

def sigmoid_backward(dA, cache):
   
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    
    return dZ


In [12]:
def linear_activation_forward(A_prev, W, b, activation):
   
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)

    return A, cache

In [13]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2  
    for l in range(1, L):
        A_prev = A 
        A, linear_activation_cache = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], "relu")
        caches.append(linear_activation_cache)
    AL, linear_activation_cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], "sigmoid")
    caches.append(linear_activation_cache)
    assert(AL.shape == (1,X.shape[1]))
    
    return AL, caches

In [14]:
def compute_cost(AL, Y):

    m = Y.shape[1]
    cost = -1 / m * (np.dot(Y, np.log(AL).T) + np.dot(1 - Y, np.log(1 - AL).T))
    cost = np.squeeze(cost)    
    assert(cost.shape == ())
    
    return cost

In [15]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = 1 / m * np.dot(dZ, A_prev.T)
    db = 1 / m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T, dZ)
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)

    return dA_prev, dW, db

In [16]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache

    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [17]:
def update_parameters(parameters, grads, learning_rate):
   
    L = len(parameters) // 2 
    for l in range(L):
        parameters["W" + str(l + 1)] -= learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] -= learning_rate * grads["db" + str(l + 1)]
    return parameters

In [18]:
def two_layer_model(X, Y, layers_dims, learning_rate, num_iterations):
    grads = {}
    costs = []                           
    m = X.shape[1]                          
    (n_x, n_h, n_y) = layers_dims

    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    for i in range(0, num_iterations):
        A1, cache1 = linear_activation_forward(X, W1, b1, "relu")
        A2, cache2 = linear_activation_forward(A1, W2, b2, "sigmoid")
        cost = compute_cost(A2, Y)
        dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))
        dA1, dW2, db2 = linear_activation_backward(dA2, cache2, "relu")
        dA0, dW1, db1 = linear_activation_backward(dA1, cache1, "sigmoid")
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
        parameters = update_parameters(parameters, grads, learning_rate);
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
    return parameters

In [25]:
n_x = 150528
n_h = 9
n_y = 1
layers_dims = (n_x, n_h, n_y)
rate = 0.007
iterations = 500

In [26]:
def predict(X, y, parameters):
    m = X.shape[1]
    n = len(parameters) // 2
    p = np.zeros((1,m))

    probas, caches = L_model_forward(X, parameters)

    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0

    print("Accuracy: "  + str(100*np.sum((p == y)/m)))

    return p

In [27]:
parameters = two_layer_model(X_train, Y_train, layers_dims, rate, iterations)

In [28]:
print("learning_rate: " + str(rate))

print("number of iterations: " + str(iterations))

print("train set:")
predictions_train = predict(X_train, Y_train, parameters)
print("test set:")
prediction_test = predict(X_test, Y_test, parameters)

learning_rate: 0.007
number of iterations: 500
train set:
Accuracy: 73.36683417085426
test set:
Accuracy: 73.54014598540145
